In [5]:
!pip install s3fs

Keyring is skipped due to an exception: 'keyring.backends'


In [3]:
import pandas as pd
import numpy as np
import fsspec

In [4]:
fsspec.filesystem('s3').invalidate_cache()

In [5]:
s3_bucket = "sagemaker-us-east-1-602900100639"
s3_key_prefix = "data/kkbox-customer-churn-model/raw"

In [10]:
train_df = pd.read_csv(f's3://{s3_bucket}/{s3_key_prefix}/train.csv')

In [11]:
train_df.shape

(992931, 2)

In [12]:
train_df.head(20)

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1
5,GBy8qSz16X5iYWD+3CMxv/Hm6OPSrXBYtmbnlRtknW0=,1
6,lYLh7TdkWpIoQs3i3o6mIjLH8/IEgMWP9r7OpsLX0Vo=,1
7,T0FF6lumjKcqEO0O+tUH2ytc+Kb9EkeaLzcVUiTr1aE=,1
8,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1
9,MkuWz0Nq6/Oq5fKqRddWL7oh2SLUSRe3/g+XmAWqW1Q=,1


# Processing Transaction Dataset

In [13]:
trans_df = pd.read_csv(f's3://{s3_bucket}/{s3_key_prefix}/transactions.csv')

In [14]:
trans_df.shape

(21547746, 9)

In [15]:
trans_df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [16]:
trans_df['transaction_date']=trans_df['transaction_date'].astype('str')
trans_df['transaction_date']=pd.to_datetime(trans_df['transaction_date'])

trans_df['membership_expire_date']=trans_df['membership_expire_date'].astype('str')
trans_df['membership_expire_date']=pd.to_datetime(trans_df['membership_expire_date'])

In [17]:
temp_transaction_cust=trans_df.groupby(['msno']).count()

In [18]:
temp_transaction_cust.head()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
msno,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,1,1,1,1,1,1,1,1
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1,1,1,1,1,1,1,1
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,4,4,4,4,4,4,4
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,19,19,19,19,19,19,19
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,26,26,26,26,26,26,26


In [20]:
temp_transaction_cust=temp_transaction_cust.reset_index()

In [21]:
temp_transaction_cust.head()

,index,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,1,1,1,1,1,1,1,1
1,1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1,1,1,1,1,1,1,1
2,2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,4,4,4,4,4,4,4
3,3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,19,19,19,19,19,19,19
4,4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,26,26,26,26,26,26,26


In [22]:
# Find the highest payment plan days
temp_payment_cust=trans_df.loc[:,['msno','payment_plan_days']]
nose=temp_payment_cust.groupby(['msno'])['payment_plan_days'].agg(lambda x: pd.Series.mode(x)[0])
temp_payment_cust=nose.reset_index()

In [23]:
temp_transaction_cust.shape

(2363626, 10)

In [24]:
temp_payment_method=trans_df.loc[:,['msno','payment_method_id']]
nose=temp_payment_method.groupby(['msno'])['payment_method_id'].agg(lambda x: pd.Series.mode(x)[0])
temp_payment_method=nose.reset_index()

In [25]:
revenue = trans_df.groupby(by='msno')['actual_amount_paid'].sum().reset_index()

In [26]:
auto_renew=trans_df.groupby(['msno'])['is_auto_renew'].agg(lambda x: pd.Series.mode(x)[0]).reset_index()

In [27]:
cancelations=trans_df.groupby(['msno'])['is_cancel'].sum().reset_index()


In [28]:
trans_df['quarter']=trans_df.transaction_date.dt.quarter
trans_df['quarter_end']=trans_df.membership_expire_date.dt.quarter

In [29]:
quarter=trans_df.groupby(['msno'])['quarter'].agg(lambda x: pd.Series.mode(x)[0]).reset_index()

In [30]:
final_df = temp_transaction_cust.loc[:,['msno', 'payment_method_id']].merge(temp_payment_method, on='msno')


In [31]:
final_df.head()

,msno,payment_method_id_x,payment_method_id_y
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,1,35
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1,38
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,41
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,39
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,41


In [32]:
final_df = final_df.merge(temp_payment_method, on='msno')

In [33]:
final_df = final_df.merge(revenue, on='msno')
final_df = final_df.merge(auto_renew, on='msno')
final_df = final_df.merge(cancelations, on='msno')
final_df = final_df.merge(quarter, on='msno')

In [34]:
final_df.shape

(2363626, 8)

In [35]:
final_df.head()

,msno,payment_method_id_x,payment_method_id_y,payment_method_id,actual_amount_paid,is_auto_renew,is_cancel,quarter
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,1,35,35,0,0,0,3
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1,38,38,1788,0,0,4
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,41,41,396,1,0,1
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,39,39,2831,1,0,3
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,41,41,3874,1,0,1


In [ ]:
final_df.columns=['msno','regist_trans','mst_frq_plan_days','mst_frq_pay_met','revenue','is_auto_renew','regist_cancels','qtr_trans']

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('data/final_transactions.csv', index=False)

# Feature Engineering For the Members dataset

In [36]:
members_df = pd.read_csv(f's3://{s3_bucket}/{s3_key_prefix}/members_v3.csv')

In [37]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [38]:
members_df.describe()

,city,bd,registered_via,registration_init_time
count,6.769473e+06,6.769473e+06,6.769473e+06,6.769473e+06
mean,3.847358e+00,9.795794e+00,5.253069e+00,2.014518e+07
std,5.478359e+00,1.792590e+01,2.361398e+00,2.318601e+04
min,1.000000e+00,-7.168000e+03,-1.000000e+00,2.004033e+07
25%,1.000000e+00,0.000000e+00,4.000000e+00,2.014042e+07
50%,1.000000e+00,0.000000e+00,4.000000e+00,2.015101e+07
75%,4.000000e+00,2.100000e+01,7.000000e+00,2.016060e+07
max,2.200000e+01,2.016000e+03,1.900000e+01,2.017043e+07


In [39]:
members_df.shape

(6769473, 6)

In [40]:
members_df['msno'].nunique()

6769473

In [41]:
members_df.isna().sum()

msno                            0
city                            0
bd                              0
gender                    4429505
registered_via                  0
registration_init_time          0
dtype: int64

In [ ]:
members_df.fillna('other', inplace=True)

In [ ]:
members_df['registration_init_time'] = pd.to_datetime(members_df['registration_init_time'])

In [ ]:
members_df.info()

In [ ]:
# members_df = members_df[ (members_df['bd'] > 0) & (members_df['bd'] < 100 )]

In [ ]:
members_df.shape

In [ ]:
members_df.to_csv('data/final_members.csv',index=False)

# Process Log Dataset

In [ ]:
# ulog_df = pd.read_csv("s3://sagemaker-us-east-2-869530972998/data/kkbox-customer-churn-model/input/user_logs_v2.csv")
ulog_df = pd.read_csv("s3://sagemaker-us-east-1-602900100639/data/kkbox-customer-churn-model/raw/user_logs.csv")

In [ ]:
ulog_df.head()

In [ ]:
ulog_df['date'] = pd.to_datetime(ulog_df['date'])

In [ ]:
ulog_df['total_secs']=np.log(ulog_df['total_secs'])

In [74]:
ulog_df=ulog_df.groupby(by='msno').mean().reset_index()

In [75]:
ulog_df.shape

(5234111, 8)

In [76]:
ulog_df.to_csv('data/final_logs.csv',index=False)

# Putting all Data Together

In [6]:
train_df = pd.read_csv(f's3://{s3_bucket}/{s3_key_prefix}/train.csv')
final_df = pd.read_csv("data/final_transactions.csv")
members_df = pd.read_csv("data/final_members.csv")
ulog_df = pd.read_csv("data/final_logs.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/final_transactions.csv'

In [11]:
final_train_df = train_df.merge(final_df, on='msno', how='left')
final_train_df = final_train_df.merge(members_df, on='msno', how='left')
final_train_df = final_train_df.merge(ulog_df, on='msno', how='left')

In [12]:
final_train_df.shape

(992931, 21)

In [13]:
final_train_df.head()

,msno,is_churn,regist_trans,mst_frq_plan_days,mst_frq_pay_met,revenue,is_auto_renew,regist_cancels,qtr_trans,city,...,gender,registered_via,registration_init_time,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,2,38,38,149,0,0,1,18.0,...,female,9.0,1970-01-01 00:00:00.020050406,1.961538,0.615385,0.730769,0.423077,15.730769,17.769231,7.944578
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,23,39,39,3458,1,2,3,10.0,...,male,9.0,1970-01-01 00:00:00.020050407,0.857965,0.301344,0.232246,0.341651,19.082534,13.880998,7.872214
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,10,39,39,1492,1,1,1,11.0,...,female,9.0,1970-01-01 00:00:00.020051016,6.624473,4.008439,1.274262,2.021097,42.333333,48.662447,8.842402
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,2,17,17,1788,0,0,1,13.0,...,female,9.0,1970-01-01 00:00:00.020051102,9.884354,3.292517,2.434014,2.223129,24.585034,25.141497,8.413886
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,8,38,38,3576,0,0,1,3.0,...,male,9.0,1970-01-01 00:00:00.020051228,1.832454,0.941953,0.899736,4.258575,90.674142,95.875989,9.884992


In [14]:
pd.DataFrame({'%MissingValues':round((final_train_df.isna().sum()/final_train_df.shape[0])*100,2)})

,%MissingValues
msno,0.00
is_churn,0.00
regist_trans,0.00
mst_frq_plan_days,0.00
mst_frq_pay_met,0.00
revenue,0.00
is_auto_renew,0.00
regist_cancels,0.00
qtr_trans,0.00
city,11.66


In [15]:
#Checking for duplicated values

print(final_train_df.shape)
print(final_train_df.msno.nunique())

(992931, 21)
992931


In [16]:
upper_thresh = 80
final_train_df.loc[final_train_df['bd'] > upper_thresh, 'bd'] = 0

In [17]:
final_train_df.bd.fillna(0,inplace=True)

In [18]:
final_train_df.dropna(axis=0,subset=['city','gender','registered_via','registration_init_time'],inplace=True)

In [19]:
final_train_df=final_train_df.astype({'mst_frq_pay_met':'object','is_auto_renew':'object','mst_frq_pay_met':'object',
           'qtr_trans':'object','city':'object','gender':'object',
           'registered_via':'object'
})
     

In [20]:
final_train_df.registration_init_time = pd.to_datetime(final_train_df.registration_init_time)

In [21]:
final_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 877161 entries, 0 to 992929
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   msno                    877161 non-null  object        
 1   is_churn                877161 non-null  int64         
 2   regist_trans            877161 non-null  int64         
 3   mst_frq_plan_days       877161 non-null  int64         
 4   mst_frq_pay_met         877161 non-null  object        
 5   revenue                 877161 non-null  int64         
 6   is_auto_renew           877161 non-null  object        
 7   regist_cancels          877161 non-null  int64         
 8   qtr_trans               877161 non-null  object        
 9   city                    877161 non-null  object        
 10  bd                      877161 non-null  float64       
 11  gender                  877161 non-null  object        
 12  registered_via          877161

In [22]:
#Create a feature that calculate tenure
import datetime

ref_date= datetime.datetime(2017, 3, 31)

final_train_df['tenure']=(ref_date-final_train_df.registration_init_time)

In [23]:
final_train_df.tenure=final_train_df.tenure.apply(lambda x: str(x).split()[0])

In [24]:
#Expressing tenure in years

final_train_df['tenure']=pd.to_numeric(final_train_df['tenure'])
final_train_df['tenure']=final_train_df['tenure']/365

In [25]:
regroup_feature = 'mst_frq_pay_met'
regroup_df = final_train_df[regroup_feature].value_counts() / final_train_df.shape[0]
filtered_regroup_df = regroup_df[regroup_df < 0.005]
final_train_df.loc[ final_train_df[regroup_feature].isin(filtered_regroup_df.index) , regroup_feature] = 'other'
print('Final categories: ',len(final_train_df[regroup_feature].value_counts()))

Final categories:  13


In [26]:
regroup_feature = 'registered_via'
regroup_df = final_train_df[regroup_feature].value_counts() / final_train_df.shape[0]
filtered_regroup_df = regroup_df[regroup_df < 0.005]
final_train_df.loc[ final_train_df[regroup_feature].isin(filtered_regroup_df.index) , regroup_feature] = 'other'
print('Final categories: ',len(final_train_df[regroup_feature].value_counts()))

Final categories:  5


In [27]:
regroup_feature = 'city'
regroup_df = final_train_df[regroup_feature].value_counts() / final_train_df.shape[0]
filtered_regroup_df = regroup_df[regroup_df < 0.005]
final_train_df.loc[ final_train_df[regroup_feature].isin(filtered_regroup_df.index) , regroup_feature] = 'other'
print('Final categories: ',len(final_train_df[regroup_feature].value_counts()))

Final categories:  18


In [28]:
# Feature transformation
temp=final_train_df.loc[:,['regist_trans','mst_frq_plan_days','revenue','regist_cancels']]
temp=np.log1p(temp)

In [29]:
feat=temp.columns
feat=['regist_trans', 'mst_frq_plan_days', 'revenue', 'regist_cancels']

In [51]:
from sklearn.model_selection import train_test_split

y=final_train_df[['is_churn']]

for_x=[x for x in final_train_df.columns if x not in['msno','is_churn','registration_init_time']]

X=final_train_df.loc[:,for_x]
X_copy = X.copy()
X=pd.get_dummies(X,drop_first=False)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
     

In [52]:
print(X_train.shape)
print(X_test.shape)

(701728, 58)
(175433, 58)


In [53]:
#Imputation
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(missing_values=np.nan,strategy='median')

#Fitting the imputer

num=['num_25','num_50','num_75','num_985','num_100','num_unq','total_secs']
imputer.fit(X_train[num])
X_train_transform=imputer.transform(X_train[num])
X_test_transform=imputer.transform(X_test[num])

#Transforming data
X_train[num]=X_train_transform
X_test[num]=X_test_transform

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [54]:
to_outlier=['regist_trans','regist_cancels','total_secs','tenure']

In [55]:
def modify_outliers_mean(data,features):

  '''
  This function modifies outliers with mean.
  First, the function detects outliers with Z-score, then calculates the mean of
  the feature without outliers,and finally, replaces outliers with the calculated
  mean.
  Parameters:
  -------------------
  data: dataset to be analyzed
  features: List of numerical features in the dataset
  '''

  to_del=[]
  for i in features:

    #Initialize null lists
    ind_upper=[]
    ind_lower=[]
    ind=[]

    #Calculate Z score
    data['Z_score']=(data[i]-data[i].mean())/data[i].std()
    print(data[(data['Z_score']>3) | (data['Z_score']<-3)].shape[0],' outliers detected for ',i)
    to_del.append(data[(data['Z_score']>3) | (data['Z_score']<-3)].shape[0])
    
    #Identified outliers
    ind=data[(data['Z_score']>3) | (data['Z_score']<-3)].index
    
    #Calculate mean to replace outlier

    mean_to_replace=data[(data['Z_score']<3) & (data['Z_score']>-3)][i].mean()
    
    #Replacing outliers
    data.loc[ind,i]=mean_to_replace

  print('total outliers modified: ',sum(to_del))
  data.drop(columns='Z_score',inplace=True)

In [56]:
modify_outliers_mean(X_train,to_outlier)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


142  outliers detected for  regist_trans


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


5936  outliers detected for  regist_cancels
11077  outliers detected for  total_secs
0  outliers detected for  tenure
total outliers modified:  17155


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [57]:
to_scale=['regist_trans','mst_frq_plan_days','revenue','regist_cancels','bd','tenure']

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

scaler.fit(X_train[to_scale])

X_train_scaled=scaler.transform(X_train[to_scale])
X_test_scaled=scaler.transform(X_test[to_scale])

X_train[to_scale]=X_train_scaled
X_test[to_scale]=X_test_scaled

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [58]:
X_train.head()

,regist_trans,mst_frq_plan_days,revenue,regist_cancels,bd,num_25,num_50,num_75,num_985,num_100,...,city_22.0,city_other,gender_female,gender_male,gender_other,registered_via_3.0,registered_via_4.0,registered_via_7.0,registered_via_9.0,registered_via_other
625221,-1.109113,0.580911,-1.258245,-0.524495,0.957116,6.529412,2.117647,1.176471,0.686275,13.176471,...,0,0,1,0,0,0,0,1,0,0
954199,0.206593,0.580911,-0.539414,1.377221,-0.807531,13.970803,1.770073,1.058394,1.605839,21.700730,...,0,0,0,0,1,0,0,1,0,0
645149,-1.707161,-0.792032,-1.526899,-0.524495,0.713716,1.394737,0.447368,0.342105,0.394737,25.684211,...,0,0,1,0,0,0,0,0,1,0
531486,-0.511065,0.580911,-0.858894,-0.524495,-0.807531,2.000000,0.266667,0.300000,0.266667,11.466667,...,0,0,0,0,1,0,0,1,0,0
200896,0.326202,0.580911,-0.299803,-0.524495,-0.807531,1.948980,0.673469,0.724490,0.602041,8.000000,...,0,0,0,0,1,0,0,1,0,0


In [59]:
X_train.to_csv('data/X_train.csv',index=False)
X_test.to_csv('data/X_test.csv',index=False)
y_train.to_csv('data/y_train.csv',index=False)
y_test.to_csv('data/y_test.csv',index=False)